# Tutorial: Workspace Manipulations
In this tutorial we demonstrate how to create a workspace and perform basic operations on it, including cropping, rotation, thresholding.

In [ ]:
# ONLY FOR GOOGLE COLAB: Run this cell only the first time you open a tutorial
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c fsemerar puma

First, we import puma. Note that in order to run pumapy, the puma conda environment must first be activated, by executing "conda activate puma" in a terminal

In [1]:
import numpy as np
import pumapy as puma
import os

A workspace is the datastructure at the basis of both PuMA and pumapy and it is basically a container for the material sample that you want to analyze. A workspace is made of little cubes, or voxels (i.e. 3D pixels), holding a value. This simple element definition (formally called Cartesian grid) allows for very fast operations. Inside a workspace object, two different arrays are defined: one called "matrix" and the other called "orientation". Both of these are nothing but a 3D Numpy array for the matrix (X,Y,Z dimensions of the domain) and a 4D Numpy array for the orientation (dimensions of X,Y,Z,3 for vectors throughout the domain). 

Next we show the different ways we have implemented to define a workspace class:

In [23]:
# defines a workspace full of zeros of shape 10x11x12
ws1 = puma.Workspace.from_shape((10, 11, 12))
print("Shape of workspace 1: {}\n".format(ws1.matrix.shape))

# defines a workspace of shape 20x31x212, full of a custom value (in this case ones)
ws2 = puma.Workspace.from_shape_value((20, 31, 212), 1)
print("Shape of workspace 2: {}\n".format(ws2.matrix.shape))

# defines a workspace of shape 5x6x2, full of a custom value (in this case ones) for the matrix array
# and vectors for the orientation array
ws3 = puma.Workspace.from_shape_value_vector((5, 6, 2), 1, (0.4, 2, 5))
print("Matrix shape of workspace 3: {}".format(ws3.matrix.shape))
print("Orientation shape of workspace 3: {}".format(ws3.orientation.shape))
print("Display Workspace 3 matrix")
ws3.show_matrix()
print("\nDisplay Workspace 3 orientation")
ws3.show_orientation()

# we can also convert a Numpy array into a Workspace directly by running:
array = np.random.randint(5, size=(10, 10, 10))
ws4 = puma.Workspace.from_array(array)
print("\nMatrix shape of workspace 4: {}".format(ws4.get_shape()))

# finally, we can also create an empty workspace object and assign its matrix directly (not recommended):
ws5 = puma.Workspace()
ws5.matrix = np.random.randint(5, size=(10, 10, 3))
print("\nDisplay Workspace 5")
ws5.show_matrix()

print("\nDifferent ways to index the matrix array:")
# N.B. the following commands are equivalent
print(ws5[0, 0, 0])
print(ws5.matrix[0, 0, 0])

Shape of workspace 1: (10, 11, 12)

Shape of workspace 2: (20, 31, 212)

Matrix shape of workspace 3: (5, 6, 2)
Orientation shape of workspace 3: (5, 6, 2, 3)
Display Workspace 3 matrix

3D Workspace:
  o---> y
  |
x v
[(:,:,0)
[[1 1 1 1 1 1]
[1 1 1 1 1 1]
[1 1 1 1 1 1]
[1 1 1 1 1 1]
[1 1 1 1 1 1]

(:,:,1)
[[1 1 1 1 1 1]
[1 1 1 1 1 1]
[1 1 1 1 1 1]
[1 1 1 1 1 1]
[1 1 1 1 1 1]]

Display Workspace 3 orientation

3D Orientation:
  o---> y
  |
x v
[(:,:,0)
[[(0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0)]
[(0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0)]
[(0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0)]
[(0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0)]
[(0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0, 5.0)]

(:,:,1)
[[(0.4, 2.0, 5.0) (0.4, 2.0, 5.0) (0.4, 2.0

It is important to keep the first three dimensions (X,Y,Z) of the matrix and orientation class variables the same. This is automatically enforced by using the class methods, but it is not when assigning them directly as in the last example. 

We can import a tomography image directly into a workspace: 

In [2]:
ws_raw = puma.import_3Dtiff(puma.path_to_example_file("200_FiberForm.tif"), 1.3e-6)

Importing /Users/fsemerar/Documents/PuMA_playground/puma-dev/python/pumapy/data/200_FiberForm.tif ... Done


The voxel length of the workspace can either be set during import of a 3D tiff, or manually afterwards, as shown below: 

In [3]:
ws_raw.voxel_length = 1.3e-6

We can visualize its slices by running the command below. By scrolling on top of the plot, you can slice through the material along the z axis. You can also use the left/right arrows on the keyboard to skip +/-10 slices or the up/down arrows to skip +/-100 slices. In addition, on the bottom of the plot, the (x,y) coordinates are shown along with the corresponding grayscale value. 
Note that in Colab only static plots are allowed, so an index can be specified to indicate the slice to show.

In [4]:
%matplotlib widget
slices = puma.plot_slices(ws_raw, slice_direction='z', crange=None, cmap='gray', index=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Next, we show how to manipulate the domain, e.g. crop, rescale, resize and rotate it. 

An approach to crop a domain is the following:

In [16]:
ws_copy = ws_raw.copy()
ws_copy.matrix = ws_copy[10:40, 35:, -20:]
print("Shape of original workspace: {}".format(ws_raw.get_shape()))
print("Shape of cropped workspace: {}".format(ws_copy.get_shape()))

Shape of original workspace: (200, 200, 200)
Shape of cropped workspace: (30, 165, 20)


However, it is important to not fall in the trap of referencing the same Numpy array. Here is an example of how you SHOULD NOT perform cropping:

In [15]:
ws_bad = puma.Workspace()
ws_bad.matrix = ws_raw[10:40, 35:, -20:]  # WRONG: always make a copy first!
ws_bad[0, 0, 0] = np.random.randint(0, 255)  # otherwise, this line also changes ws_raw
print(ws_raw[10, 35, -20])
print(ws_bad[0, 0, 0])

228
228


As you can see from the output, both the original Workspace and the newly created one share the same Numpy array for the matrix class variable (the second one is only a section of it). This way, when one is changed, the other one will be changed as well. It is important to make a copy of a domain if the original workspace needs to be kept.

Next, we show how we can rescale a domain by a factor or resize it to a specified size. 

In [18]:
ws_copy = ws_raw.copy()
ws_copy.rescale(scale=0.5, segmented=False)

# Notice that now the axis have different limits
puma.compare_slices(ws_raw, ws_copy, index=50)

Rescaled workspace size: (100, 100, 100)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …